In [ ]:
from IPython import get_ipython
if 'google.colab' in str(get_ipython()):
  # instalar drivers
  !apt-get update -y
  !apt-get install -y chromium-chromedriver
  !cp  /usr/lib/chromium-browser/chromedriver /usr/bin
  !pip install selenium

In [124]:
from selenium import webdriver
from bs4 import BeautifulSoup
import time
from selenium.webdriver.common.by import By
from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.support import expected_conditions
from selenium.webdriver.support.wait import WebDriverWait
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.desired_capabilities import DesiredCapabilities
if 'google.colab' in str(get_ipython()):
  # instalar drivers
  options = webdriver.ChromeOptions()
  options.add_argument('-headless')
  options.add_argument('-no-sandbox')
  options.add_argument('-disable-dev-shm-usage')
  options.add_argument('--user-agent=""Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/74.0.3729.157 Safari/537.36""') # user agent
  #options.headless = True
  #options.add_argument("--window-size=1920,1200")
  # Instanciamos el webdriver
  driver = webdriver.Chrome('chromedriver',options=options)
else:
  import os 
  os.environ["PATH"] = '$PATH:.'
  #driver = webdriver.Chrome('C:/Users/usuari/Documents/masterM2/Data_Digital/chromedriver')
  driver = webdriver.Firefox()

In [125]:
#define url path 
driver.get('https://www.imdb.com/chart/moviemeter/')

In [126]:
soup = BeautifulSoup(driver.page_source)

In [127]:
url_movie=[]
list_href = soup.find_all('td', class_ ="posterColumn")

for item in list_href:
  part_href= item.find('a').get('href')
  url_movie.append('https://www.imdb.com{}'.format(part_href))

print(url_movie[5])

https://www.imdb.com/title/tt1877830/?pf_rd_m=A2FGELUUNOQJNL&pf_rd_p=ea4e08e1-c8a3-47b5-ac3a-75026647c16e&pf_rd_r=VY05NHWXS3A6C9T63ZDD&pf_rd_s=center-1&pf_rd_t=15506&pf_rd_i=moviemeter&ref_=chtmvm_tt_6


In [67]:
driver.get('https://www.imdb.com/title/tt4123432/?pf_rd_m=A2FGELUUNOQJNL&pf_rd_p=ea4e08e1-c8a3-47b5-ac3a-75026647c16e&pf_rd_r=6W51KPC95P44E8H2CT85&pf_rd_s=center-1&pf_rd_t=15506&pf_rd_i=moviemeter&ref_=chtmvm_tt_2')

In [128]:
len(url_movie)

100

In [135]:
dict_all_values = {}
position= 0

for url in range(0,len(url_movie)-94):

    position += position + 1
    driver.get(url_movie[url])
    time.sleep(4)

    #obtener nombre de pelicula
    name_movie = driver.find_element(By.XPATH,'//div[@class="sc-94726ce4-1 iNShGo"]/h1').text

    #obtener calificacion de la pelicula
    list_rate = driver.find_elements(By.XPATH,'//div[@class="sc-7ab21ed2-2 kYEdvH"]/span')
    rate_movie = ''
    for rate in list_rate:
      rate_movie += rate.text
      
     #obtener fecha, año, calificacion
    path_data = driver.find_elements(By.XPATH,'//div[@class="sc-94726ce4-3 eSKKHi"]/ul//li[@class="ipc-inline-list__item"]')
    year_movie= path_data[0].text
    parental_guide= path_data[1].text
    duration_movie= path_data[2].text

    #obtener la descripcion de la pelicula
    descripcion = driver.find_element(By.XPATH,'//p[@class="sc-16ede01-6 cXGXRR"]').text
    #print(descripcion)

    #obtener lista de  generos  de la pelicula
    genders = driver.find_element(By.XPATH,'//div[@class="ipc-chip-list sc-16ede01-4 bMBIRz"]').text
    #print()

    #print (name_movie, rate_movie,year_movie,parental_guide,duration_movie,genders)

    #Define name dictinary by movies
    dict_all_values[name_movie]={"rating_movie":[],
                                          "duration_movie":[],
                                          "year_movie":[],
                                          "description_movie":[],
                                          "genders_movie":[]}

    dict_all_values[name_movie]['rating_movie'].append(rate_movie)
    dict_all_values[name_movie]['duration_movie'].append(duration_movie)
    dict_all_values[name_movie]['year_movie'].append(year_movie)
    dict_all_values[name_movie]['description_movie'].append(descripcion)
    dict_all_values[name_movie]['genders_movie'].append(genders.replace('\n', ','))



In [136]:
import json
print(json.dumps(dict_all_values, sort_keys=False, indent=4))

{
    "The Northman": {
        "rating_movie": [
            "7.9/10"
        ],
        "duration_movie": [
            "2h 16m"
        ],
        "year_movie": [
            "2022"
        ],
        "description_movie": [
            "From visionary director Robert Eggers comes The Northman, an action-filled epic that follows a young Viking prince on his quest to avenge his father's murder."
        ],
        "genders_movie": [
            "Action,Adventure,Drama"
        ]
    },
    "Fantastic Beasts: The Secrets of Dumbledore": {
        "rating_movie": [
            "6.6/10"
        ],
        "duration_movie": [
            "2h 22m"
        ],
        "year_movie": [
            "2022"
        ],
        "description_movie": [
            "Albus Dumbledore assigns Newt and his allies with a mission related to the rising power of Grindelwald."
        ],
        "genders_movie": [
            "Adventure,Family,Fantasy"
        ]
    },
    "Everything Everywhere All at Once":

# **Download files JSON**

In [137]:
from IPython.core.display import JSON
# load json module
import json
import codecs
from google.colab import files

# create json object from dictionary
json = json.dumps(dict_all_values)

# open file for writing, "w" 
f = open("dict_movie.json","w",encoding='utf-8')

# write json object to file
f.write(json)

# close file
f.close()

files.download('dict_movie.json')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

# **PASS DICT TO DATAFRAME**

In [148]:
import pandas as pd
df = pd.DataFrame.from_dict(dict_all_values, orient="index").reset_index()


In [149]:
print(df.head(5))

                                         index rating_movie duration_movie  \
0                                 The Northman     [7.9/10]       [2h 16m]   
1  Fantastic Beasts: The Secrets of Dumbledore     [6.6/10]       [2h 22m]   
2            Everything Everywhere All at Once     [8.9/10]       [2h 19m]   
3                            Death on the Nile     [6.3/10]        [2h 7m]   
4                                            X     [6.9/10]       [1h 45m]   

  year_movie                                  description_movie  \
0     [2022]  [From visionary director Robert Eggers comes T...   
1     [2022]  [Albus Dumbledore assigns Newt and his allies ...   
2     [2022]  [An aging Chinese immigrant is swept up in an ...   
3     [2022]  [While on vacation on the Nile, Hercule Poirot...   
4     [2022]  [In 1979, a group of young filmmakers set out ...   

                genders_movie  
0    [Action,Adventure,Drama]  
1  [Adventure,Family,Fantasy]  
2   [Action,Adventure,Comedy]  
